In [306]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import matplotlib
import re
from datetime import datetime
from wordcloud import WordCloud 
import jieba
import jieba.analyse
from datetime import date
import re
import string
from wordcloud import WordCloud 

### 構面字典

In [307]:
# dict_ = pd.read_csv('../構面字典-new.csv')
dict_ = pd.read_csv('./aspect.csv')
df_dict = dict_[['name','alias']]
d_list = df_dict.to_dict(orient='records')
structure_dict = {} # 構面字典 => name:alias
for i in d_list:
    structure_dict[i['name']] = i['alias']
structure_dict.keys()#key:成就感,...

dict_keys(['成就感', '學習成長', '創新', '薪資', '福利', '管理制度', '工作氛圍', '同事互動', '主管風格', '工作地點', '公司規模', '工作環境', '產業前景', '輪調', '外派', '出差', '引擎', '馬力', '避震', '外觀', '操控', '安全', '堅固', '配備', '價錢', '科技', '品質', '折舊', '品牌', '空間', '保養', '續航', '尺寸', '車種', '驅動', '變速箱', '電資', '機械', '製造', '車輛工程', '品管', '多元性', '企業社會責任', '企業永續目標', '面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力', '招募人員', '正向', '負向'])

In [308]:
structure_origin = dict_[['class','name']]
structures_list = structure_origin['class'].unique().tolist() # 有哪些構面字陣列

structure_origin_list = pd.DataFrame(structure_origin.groupby('class')['name'].apply(list)).reset_index().to_dict(orient='records')
transformed_data = [{item['class']: item['name']} for item in structure_origin_list] # 取得構面字典，型態為list of dict

In [309]:
new_dict = {}
for i in transformed_data:
    new_dict[list(i.keys())[0]] = list(i.values())[0]
new_dict

{'人際價值': ['工作氛圍', '同事互動', '主管風格'],
 '公司文化與價值': ['多元性', '企業社會責任', '企業永續目標'],
 '情緒': ['正向', '負向'],
 '成長價值': ['成就感', '學習成長', '創新'],
 '招募人員': ['招募人員'],
 '招募資訊': ['面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力'],
 '產品與服務': ['引擎',
  '馬力',
  '避震',
  '外觀',
  '操控',
  '安全',
  '堅固',
  '配備',
  '價錢',
  '科技',
  '品質',
  '折舊',
  '品牌',
  '空間',
  '保養',
  '續航',
  '尺寸',
  '車種',
  '驅動',
  '變速箱'],
 '科系專業': ['電資', '機械', '製造', '車輛工程', '品管'],
 '組織安全與安定': ['工作地點', '公司規模', '工作環境', '產業前景'],
 '經濟報酬': ['薪資', '福利', '管理制度'],
 '變化性': ['輪調', '外派', '出差']}

In [310]:
# 以pattern_text 例如是成就感，那就會去找text中有出現成就感的alias的次數
def get_dict_num(pattern_text, text):
    pattern = r"(" + structure_dict[pattern_text] + ")"
    num = len(re.findall(pattern, text))
    return num

In [311]:
# 取得資料
df_ = pd.read_csv('./interview_clean.csv')
df_.drop(['Unnamed: 0','rank','difficult'], axis=1,inplace=True)
df_.rename(columns={'whole_art':'content'}, inplace=True)
df_ = df_[~df_['process'].isna()].reset_index(drop='index') # (7219 , 8)
df_.head()
df_
# df_ = df_[~df_['month_salary'].isna()].reset_index(drop='index')
# df_.drop(['month_salary','year_salary','year-end_salary','bonus','dividends','other','average_day','seniority','total_seniority','mood'], axis = 1, inplace=True)

,aID,company_name,vacancies,post_time,interview_time,status,process,question,suggestion,content,word
0,0,裕隆集團_納智捷汽車股份有限公司,企劃管理師,2022.08.24,2022.07,未錄取,針對專業提出疑問，並...,"[{'Q': '這個品牌業界傳的都不太好，為什麼願意來納智捷面試', 'A': '因為我本來...",NaN,針對專業提出疑問，並。這個品牌業界傳的都不太好，為什麼願意來納智捷面試？因為我本來就愛納智捷。,"['專業', '提出', '疑問', '品牌', '業界', '願意', '納智捷', '面..."
1,1,裕隆集團_納智捷汽車股份有限公司,管理師,2022.02.21,2019.02,無聲卡,整體面試過程約2小時...,"[{'Q': '對於納智捷的了解', 'A': '依照職位跟公司官網做...'}]",NaN,整體面試過程約2小時。對於納智捷的了解？依照職位跟公司官網做。,"['體面', '過程', '小時', '納智捷', '職位', '公司', '官網']"
2,2,裕隆集團_納智捷汽車股份有限公司,行銷專員,2022.01.13,2017.09,錄取,第一次面試先簡單自我...,"[{'Q': '會不會開車', 'A': '有駕照，但當時還不熟...'}]",NaN,第一次面試先簡單自我。會不會開車？有駕照，但當時還不熟。,"['面試', '簡單', '自我', '會不會', '開車', '駕照', '熟']"
3,3,裕隆集團_納智捷汽車股份有限公司,人力資源管理師,2021.09.12,2021.09,未錄取,第一關，人資面試\n...,"[{'Q': '結算薪資怎麼做', 'A': '照実回答'}]",NaN,第一關，人資面試。。結算薪資怎麼做？照実回答。,"['人資', '面試', '結算', '薪資', '照実', '回答']"
4,4,裕隆集團_納智捷汽車股份有限公司,秘書,2021.04.21,2017.09,錄取,在台南東區的廠二樓面...,"[{'Q': '與人交談方面的問題', 'A': '我是個開朗的人，比較...'}]",NaN,在台南東區的廠二樓面。與人交談方面的問題？我是個開朗的人，比較。,"['台南', '東區', '二樓面', '交談', '方面', '問題', '開朗']"
...,...,...,...,...,...,...,...,...,...,...,...
7249,7636,匯豐汽車股份有限公司,產品企劃專員,2018.04.20,2017.12,錄取,簡報說明令人印象深刻,"[{'Q': '針對產品提出看法', 'A': '直接腦袋中的想法講出...'}]",NaN,簡報說明令人印象深刻。針對產品提出看法？直接腦袋中的想法講出。,"['簡報', '說明', '令', '印象深刻', '產品', '提出', '看法', '直..."
7250,7637,匯豐汽車股份有限公司,SUM服務專員,2018.03.23,2010.06,錄取,面試過程其實很好，而...,"[{'Q': '時間有點久不太記得，但是要看你遇到的廠長是好是壞!', 'A': '時間有點...",NaN,面試過程其實很好，而。時間有點久不太記得，但是要看你遇到的廠長是好是壞!？時間有點久不太記得，。,"['面試', '過程', '時間', '記得', '遇到', '廠長', '壞', '時間'..."
7251,7638,匯豐汽車股份有限公司,總公司-品牌部,2018.01.24,2016.06,錄取,帶到會議室，等待面試...,[{'Q': '對於行銷與品牌的各種看法，EXCEL操作技能，網站經營(GA)，網路行銷規劃...,NaN,帶到會議室，等待面試。對於行銷與品牌的各種看法，EXCEL操作技能，網站經營(GA)，網路行...,"['帶到', '會議室', '等待', '面試', '行銷', '品牌', '看法', 'E..."
7252,7639,匯豐汽車股份有限公司,技師,2017.12.24,2002.06,錄取,直接面試 問維修相關...,"[{'Q': '需配合值班可否接受 前份工作為何離職', 'A': '可以\n\n職涯規劃'}]",NaN,直接面試 問維修相關。需配合值班可否接受 前份工作為何離職？可以。職涯規劃。,"['直接面試問', '維修', '相關', '需', '配合', '值班', '可否', '..."


In [312]:
# 只用有解鎖的文章
# df_open = df_[df_['process'].apply(lambda x: isopen(x))].reset_index(drop='index')
df_open = df_[~df_['suggestion'].isna()]
df_open.shape

(536, 11)

In [313]:
# 字典型態的資料
# 資料轉為字典，比較方便去做計算
dict_ = df_open.to_dict(orient='records') 

In [314]:
# 取得根據每個構面下的類別名稱，計算該類別名稱出現多少次，來去計算dtm
for structure in list(structure_dict.keys()):
    for data in dict_:
        data[structure] = get_dict_num(structure, data['content'])

In [315]:
# DTM for 趨勢圖
# 把用不到的欄位踢除
kick_off_key = ['company_name','status','process','question','suggestion','frequency_overtime','Q_content']
data_list = [{k: v for k, v in d.items() if k not in kick_off_key} for d in dict_]
# 得到DTM DataFrame
dtm_df = pd.DataFrame(data_list)
dtm_df.shape

(536, 63)

In [316]:
# DTM for correlation & co-ocurence #把用不到的欄位踢除
kick_off_ = ['vacancies','post_time','interview_time','content','company_name','status','process','question','suggestion','frequency_overtime','Q_content']
co_ = [{k: v for k, v in d.items() if k not in kick_off_} for d in dict_]
dtm_co = pd.DataFrame(co_)
dtm_co.head()

,aID,word,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,41,"['專業', '能力', '測驗', '分鐘', '英文', '多益', '測驗', '分鐘...",0,0,0,0,0,2,0,0,...,1,0,0,0,0,0,6,3,2,218
1,42,"['浪費', '時間', '面試', '大夜班', '現場', '測試', '聽力', '打...",0,0,0,3,1,1,0,0,...,1,0,0,0,0,0,0,1,2,241
2,43,"['主管', '溫文儒雅', '先生', '感覺', '誠懇', '問題', '深入', '...",0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,2,3,192
3,44,"['面試', '視訊', '面試', 'HR', '物料', '管理', '部門', '主管...",0,2,0,0,0,5,0,1,...,2,0,0,0,0,0,4,3,12,647
4,45,"['遠', '端考', '英文', '專業', '測驗', '小時', '視訊', '面試'...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,1,189


In [317]:
dtm_df.head()

,aID,vacancies,post_time,interview_time,content,word,成就感,學習成長,創新,薪資,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,41,職員,2023.03.24,2023.03,首先會先做專業能力測驗，大約50分鐘，再來會做英文多益測驗40分鐘，做完後就進行視訊面試，1...,"['專業', '能力', '測驗', '分鐘', '英文', '多益', '測驗', '分鐘...",0,0,0,0,...,1,0,0,0,0,0,6,3,2,218
1,42,客服,2023.03.24,2023.01,覺得很浪費時間，我面試大夜班，到現場要先測試聽力打字，現場很吵，然後反應也沒用，也不知道在測...,"['浪費', '時間', '面試', '大夜班', '現場', '測試', '聽力', '打...",0,0,0,3,...,1,0,0,0,0,0,0,1,2,241
2,43,職員,2023.03.06,2018.01,主管是一個溫文儒雅的先生 真的感覺得出來他非常誠懇 然後問的問題沒有太多深入的題目 主要就是...,"['主管', '溫文儒雅', '先生', '感覺', '誠懇', '問題', '深入', '...",0,0,0,0,...,0,0,0,0,0,0,0,2,3,192
3,44,物料管理實習生,2023.03.02,2022.05,面試是以視訊面試進行，除了HR以外還有一個物料管理部門的主管（？）之類的長官，整體面試進行的...,"['面試', '視訊', '面試', 'HR', '物料', '管理', '部門', '主管...",0,2,0,0,...,2,0,0,0,0,0,4,3,12,647
4,45,職員,2023.02.16,2021.06,先遠端考英文及專業測驗，大約1個半小時，而後開始視訊面試。面談大約進行一小時，面試官有3-4...,"['遠', '端考', '英文', '專業', '測驗', '小時', '視訊', '面試'...",0,0,0,0,...,0,0,0,0,0,0,2,2,1,189


In [318]:
def get_ym(time):
    l = time.split('.')
    return l[0]+"-"+l[1]

In [319]:
# # 新增年月日欄位
# date_format = '%Y.%m.%d'
# dtm_df['post_time'] = dtm_df['post_time'].apply(lambda x: datetime.strptime(x, date_format))
# dtm_df['p_year'] = dtm_df['post_time'].apply(lambda x: x.year)
# dtm_df['p_month'] = dtm_df['post_time'].apply(lambda x: x.month)
# dtm_df['p_day'] = dtm_df['post_time'].apply(lambda x: x.day)
dtm_df['p_year_month'] = dtm_df['post_time'].apply(lambda x: get_ym(x))
dtm_df = dtm_df.reindex(columns=['vacancies', 'post_time', 'p_year_month', 'interview_time', 'content', 'word', '成就感', '學習成長',
       '創新', '薪資', '福利', '管理制度', '工作氛圍', '同事互動', '主管風格', '工作地點', '公司規模',
       '工作環境', '產業前景', '輪調', '外派', '出差', '引擎', '馬力', '避震', '外觀', '操控', '安全',
       '堅固', '配備', '價錢', '科技', '品質', '折舊', '品牌', '空間', '保養', '續航', '尺寸', '車種',
       '驅動', '變速箱', '電資', '機械', '製造', '車輛工程', '品管', '多元性', '企業社會責任', '企業永續目標',
       '面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力', '招募人員',
       '正向', '負向'])
dtm_df

,vacancies,post_time,p_year_month,interview_time,content,word,成就感,學習成長,創新,薪資,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,職員,2023.03.24,2023-03,2023.03,首先會先做專業能力測驗，大約50分鐘，再來會做英文多益測驗40分鐘，做完後就進行視訊面試，1...,"['專業', '能力', '測驗', '分鐘', '英文', '多益', '測驗', '分鐘...",0,0,0,0,...,1,0,0,0,0,0,6,3,2,218
1,客服,2023.03.24,2023-03,2023.01,覺得很浪費時間，我面試大夜班，到現場要先測試聽力打字，現場很吵，然後反應也沒用，也不知道在測...,"['浪費', '時間', '面試', '大夜班', '現場', '測試', '聽力', '打...",0,0,0,3,...,1,0,0,0,0,0,0,1,2,241
2,職員,2023.03.06,2023-03,2018.01,主管是一個溫文儒雅的先生 真的感覺得出來他非常誠懇 然後問的問題沒有太多深入的題目 主要就是...,"['主管', '溫文儒雅', '先生', '感覺', '誠懇', '問題', '深入', '...",0,0,0,0,...,0,0,0,0,0,0,0,2,3,192
3,物料管理實習生,2023.03.02,2023-03,2022.05,面試是以視訊面試進行，除了HR以外還有一個物料管理部門的主管（？）之類的長官，整體面試進行的...,"['面試', '視訊', '面試', 'HR', '物料', '管理', '部門', '主管...",0,2,0,0,...,2,0,0,0,0,0,4,3,12,647
4,職員,2023.02.16,2023-02,2021.06,先遠端考英文及專業測驗，大約1個半小時，而後開始視訊面試。面談大約進行一小時，面試官有3-4...,"['遠', '端考', '英文', '專業', '測驗', '小時', '視訊', '面試'...",0,0,0,0,...,0,0,0,0,0,0,2,2,1,189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,引電技師,2022.12.04,2022-12,2021.04,一開始課長會先給你一份試卷，裡面包含了汽車原理，專有名詞英翻中，繼電器原理，與實境題目，考完...,"['課長', '試卷', '包含', '汽車', '原理', '專有名詞', '英翻中', ...",0,0,0,2,...,2,0,0,0,0,0,0,0,6,279
532,教育訓練講師,2023.04.09,2023-04,2021.07,筆試時間大約兩個小時。包含英文測驗、過去在學的經歷等等，比如過去做的專題、遇到困難時要怎麼辦...,"['筆試', '時間', '小時', '包含', '英文', '測驗', '過去', '在學...",0,1,0,2,...,1,0,0,0,0,0,1,2,6,228
533,專員,2023.03.26,2023-03,2021.12,第一一關進行筆試，主要是考英文，英文建議有850以上的實力會比較好作答。第二關個案分析，給你...,"['第一一', '筆試', '英文', '英文', '建議', '850', '實力', '...",0,1,0,1,...,0,0,0,0,0,0,1,2,4,210
534,會計助理,2023.03.01,2023-03,2020.09,工作環境很好，畢竟以車業來說，公司環境也算是門面，面試有筆試完後會計主管會面談，談話過程還算...,"['工作環境', '車業', '公司', '環境', '門面', '面試', '筆試', '...",0,0,0,1,...,3,0,0,0,0,0,1,2,3,200


In [320]:
dtm_df.to_csv('dtm_result.csv', encoding='utf-8')

### 計算correlation

In [18]:
df_correlation = pd.DataFrame(np.corrcoef(dtm_co.T),columns=dtm_co.columns, index=dtm_co.columns)

/Users/changchenyu/opt/anaconda3/envs/sma/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/changchenyu/opt/anaconda3/envs/sma/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [20]:
df_corre_vect = df_correlation.reset_index()
df_corre_vect.rename(columns={'index': 'item1'}, inplace=True)
df_corre_vect

,item1,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,成就感,1.000000,0.043843,-0.012927,0.081711,-0.046301,-0.031028,-0.026186,-0.007492,-0.043460,...,0.089708,0.047853,-0.013716,-0.001807,0.014914,-0.039775,-0.034507,-0.038126,0.126932,-0.005064
1,學習成長,0.043843,1.000000,0.060496,-0.040755,0.022032,0.048898,0.008600,0.175008,0.195431,...,0.203451,-0.002673,0.028902,0.252897,0.013700,0.072116,0.202757,0.305837,0.273331,0.328328
2,創新,-0.012927,0.060496,1.000000,-0.028549,-0.029144,0.030081,-0.034661,0.108407,-0.059742,...,-0.001728,-0.022267,-0.006440,-0.017158,-0.014795,-0.018676,0.118290,-0.021991,0.128881,0.044477
3,薪資,0.081711,-0.040755,-0.028549,1.000000,0.584063,0.193192,0.058120,-0.052762,0.002311,...,0.161982,0.152310,-0.030292,0.023903,0.131384,0.013005,-0.002978,0.016203,0.133677,0.170845
4,福利,-0.046301,0.022032,-0.029144,0.584063,1.000000,0.224522,0.074269,-0.050447,-0.033819,...,0.141007,0.041224,-0.030924,0.008752,-0.016902,0.017714,-0.002645,-0.005324,0.222602,0.158065
5,管理制度,-0.031028,0.048898,0.030081,0.193192,0.224522,1.000000,0.114911,0.071079,0.065338,...,0.263214,0.027626,0.005598,-0.048423,0.079368,0.024955,0.025430,0.104163,0.108575,0.270217
6,工作氛圍,-0.026186,0.008600,-0.034661,0.058120,0.074269,0.114911,1.000000,0.238897,-0.014182,...,0.270494,-0.005949,-0.036778,0.014189,0.174460,-0.028003,0.176209,0.090415,0.258276,0.303527
7,同事互動,-0.007492,0.175008,0.108407,-0.052762,-0.050447,0.071079,0.238897,1.000000,0.040301,...,0.221062,-0.026020,0.131075,-0.028701,0.065159,0.018429,0.521688,0.098716,0.241474,0.361233
8,主管風格,-0.043460,0.195431,-0.059742,0.002311,-0.033819,0.065338,-0.014182,0.040301,1.000000,...,0.176377,-0.022391,0.117925,0.008405,0.170304,0.017707,0.154486,0.834510,0.321619,0.457814
9,工作地點,-0.042637,0.025314,-0.020020,0.133042,0.083420,0.059112,0.023088,-0.026862,0.131366,...,0.121919,0.129085,-0.021243,-0.056594,0.035786,-0.046347,0.050391,0.141632,0.094732,0.206748


In [22]:
corr_melt_df = pd.melt(df_corre_vect, id_vars='item1', value_vars=list(df_corre_vect.columns)[1:], var_name='item2', value_name='correlation')
corr_melt_df.rename(columns={'index':'item1'},inplace=True)
corr_melt_df

,item1,item2,correlation
0,成就感,成就感,1.000000
1,學習成長,成就感,0.043843
2,創新,成就感,-0.012927
3,薪資,成就感,0.081711
4,福利,成就感,-0.046301
...,...,...,...
3244,出路,負向,0.022262
3245,能力,負向,0.442509
3246,招募人員,負向,0.624562
3247,正向,負向,0.578160


In [49]:
new_dict

{'人際價值': ['工作氛圍', '同事互動', '主管風格'],
 '公司文化與價值': ['多元性', '企業社會責任', '企業永續目標'],
 '情緒': ['正向', '負向'],
 '成長價值': ['成就感', '學習成長', '創新'],
 '招募人員': ['招募人員'],
 '招募資訊': ['面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力'],
 '產品與服務': ['引擎',
  '馬力',
  '避震',
  '外觀',
  '操控',
  '安全',
  '堅固',
  '配備',
  '價錢',
  '科技',
  '品質',
  '折舊',
  '品牌',
  '空間',
  '保養',
  '續航',
  '尺寸',
  '車種',
  '驅動',
  '變速箱'],
 '科系專業': ['電資', '機械', '製造', '車輛工程', '品管'],
 '組織安全與安定': ['工作地點', '公司規模', '工作環境', '產業前景'],
 '經濟報酬': ['薪資', '福利', '管理制度'],
 '變化性': ['輪調', '外派', '出差']}

In [55]:
def get_class_by_subclass(sub_class):
    for k, v in new_dict.items():
        if (sub_class in v):
            return k

In [58]:
corr_melt_df2 = corr_melt_df[~corr_melt_df['correlation'].isnull()].reset_index(drop='index')
corr_melt_df2['class'] = corr_melt_df2['item1'].apply(lambda x: get_class_by_subclass(x))
corr_melt_df2

,item1,item2,correlation,class
0,成就感,成就感,1.000000,成長價值
1,學習成長,成就感,0.043843,成長價值
2,創新,成就感,-0.012927,成長價值
3,薪資,成就感,0.081711,經濟報酬
4,福利,成就感,-0.046301,經濟報酬
...,...,...,...,...
2396,出路,負向,0.022262,招募資訊
2397,能力,負向,0.442509,招募資訊
2398,招募人員,負向,0.624562,招募人員
2399,正向,負向,0.578160,情緒


In [70]:
corr_melt_df3 = corr_melt_df2[(corr_melt_df2['correlation'] <0) & (corr_melt_df2['correlation'] < 100)]
corr_melt_df3

,item1,item2,correlation,class
2,創新,成就感,-0.012927,成長價值
4,福利,成就感,-0.046301,經濟報酬
5,管理制度,成就感,-0.031028,經濟報酬
6,工作氛圍,成就感,-0.026186,人際價值
7,同事互動,成就感,-0.007492,人際價值
...,...,...,...,...
2338,企業永續目標,正向,-0.019633,公司文化與價值
2352,成就感,負向,-0.005064,成長價值
2369,外觀,負向,-0.006485,產品與服務
2379,車種,負向,-0.005932,產品與服務


In [67]:
a = list(corr_melt_df2['class'].unique())
a

['成長價值',
 '經濟報酬',
 '人際價值',
 '組織安全與安定',
 '變化性',
 '產品與服務',
 '科系專業',
 '公司文化與價值',
 '招募資訊',
 '招募人員',
 '情緒']

In [68]:
a.sort()
a

['人際價值',
 '公司文化與價值',
 '情緒',
 '成長價值',
 '招募人員',
 '招募資訊',
 '產品與服務',
 '科系專業',
 '組織安全與安定',
 '經濟報酬',
 '變化性']

In [59]:
corr_melt_df2.to_csv('interview_corr.csv', encoding='utf-8')

In [176]:
corr_melt_df[corr_melt_df['item1'] == '負向'].sort_values(['correlation'], ascending=False)[1:7]

,item1,item2,correlation
1025,負向,馬力,0.999949
2564,負向,面試,0.678857
3134,負向,招募人員,0.624562
3191,負向,正向,0.578160
512,負向,主管風格,0.457814
2735,負向,工作,0.454058


### 計算 co-occurrence

In [178]:
result = np.dot(dtm_co.T.values, dtm_co.values)
co_occurrence_df = pd.DataFrame(result, columns=dtm_co.columns, index=dtm_co.columns)
co_occurrence_df.head()

,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,工作地點,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
成就感,19,9,0,14,1,9,3,4,19,0,...,38,3,0,1,1,0,12,32,77,4169
學習成長,9,338,4,54,51,182,56,105,448,24,...,397,17,2,40,9,22,336,741,790,72241
創新,0,4,6,0,0,6,0,6,1,0,...,6,0,0,0,0,0,16,8,30,1596
薪資,14,54,0,592,243,303,74,32,318,49,...,414,43,0,16,23,16,206,498,716,66516
福利,1,51,0,243,248,213,52,22,187,25,...,259,16,0,9,4,11,136,311,543,42820


In [181]:
df_ocurr_vect = co_occurrence_df.reset_index()
df_ocurr_vect.rename(columns={'index': 'item1'}, inplace=True)
df_ocurr_vect
ocurr_melt_df = pd.melt(df_ocurr_vect, id_vars='item1', value_vars=list(df_ocurr_vect.columns)[1:], var_name='item2', value_name='ocurrelation')
ocurr_melt_df.rename(columns={'index':'item1'},inplace=True)
ocurr_melt_df

,item1,item2,ocurrelation
0,成就感,成就感,19
1,學習成長,成就感,9
2,創新,成就感,0
3,薪資,成就感,14
4,福利,成就感,1
...,...,...,...
3244,出路,負向,10597
3245,能力,負向,221842
3246,招募人員,負向,482379
3247,正向,負向,525814
